### Read training, dev and unlabeled test data

The following provides a starting code (Python 3) of how to read the labeled training and dev cipher text, and unlabeled test cipher text, into lists.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import WhitespaceTokenizer() method from nltk
from nltk.tokenize import WhitespaceTokenizer, WordPunctTokenizer
	
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()



In [ ]:
train, dev, test = [], [], []
train_text, dev_text, test_text = [], [], []
train_label, dev_label, test_label = [], [], []

In [ ]:
for x in open('/content/drive/MyDrive/CSCI-544/train_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    train.append(x)
    train_text.append(x[1].lower())
    train_label.append(x[0])
print (len(train))
print (train[:3])

In [ ]:
for x in open('/content/drive/MyDrive/CSCI-544/dev_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    dev.append(x)
    dev_text.append(x[1].lower())
    dev_label.append(x[0])
print (len(dev))
print (dev[:3])

#### Different from 'train' and 'dev' that are both list of tuples, 'test' will be just a list.

In [ ]:
for x in open('/content/drive/MyDrive/CSCI-544/test_enc_unlabeled.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').lower()
    test.append(x)
print (len(test))
print (test[:3])

#### You can split every sentence into lists of words by white spaces.

In [ ]:
train_split = [[x[0], x[1].lower().split(' ')] for x in train]
dev_split = [[x[0], x[1].lower().split(' ')] for x in dev]
test_split = [[x.split(' ')] for x in test]
#test_split

### Main Code Body

You may choose to experiment with different methods using your program. However, you need to embed the training and inference processes at here. We will use your prediction on the unlabeled test data to grade, while checking this part to understand how your method has produced the predictions.

Training of Embeddings happens here

In [ ]:

def process_text(list_sentences, lower=False):
    comments = []
    for text in list_sentences:
        #print(text)
        single_sentence = ' '.join(text).strip().lower()
        temp_text = single_sentence.replace('.', '')
        text = temp_text.replace(',', '')
        txt = tk.tokenize(text)
        comments.append(txt)
    return comments

sentences = []
for a, b in train_split:
  #print(b)
  sentences.append(b)
  # if 'úol' in b:
  #   print("There")
for a, b in dev_split:
  sentences.append(b)
  # if 'úol' in b:
  #   print("There")
for a in test_split:
  #print(a[0])
  sentences.append(a[0])
  # if 'úol' in a:
  #   print("There")
#print(sentences[1])
#print(sentences[-1])

new_sentence = process_text(sentences)


In [ ]:
print(len(new_sentence))

In [ ]:
from gensim.models import Word2Vec
# define training data
# train model
model = Word2Vec(new_sentence, window=5,min_count=5,negative=15,iter=10)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)
# access vector for one word
print(model['cêêö'])



In [ ]:
print("Number of word vectors: {}".format(len(model.wv.vocab)))
print(model.wv.vocab)


In [ ]:
MAX_NB_WORDS = len(model.wv.vocab)
MAX_SEQUENCE_LENGTH = 100
word_index = {t: i+1 for i,t in enumerate(model.wv.vocab)}

print(len(word_index))

Fasttext


In [ ]:
from gensim.models import FastText
model2 = FastText(size=100, window=3, min_count=1, sentences=new_sentence, iter=10)
print("Number of word vectors: {}".format(len(model2.wv.vocab)))
print(model2.wv.vocab)

In [ ]:
MAX_NB_WORDS = len(model2.wv.vocab)
MAX_SEQUENCE_LENGTH = 100
word_index_2 = {t: i+1 for i,t in enumerate(model2.wv.vocab)}

print(len(word_index_2))
print(model2['lkêcê'])

Actual work with data

In [ ]:
# from nltk.probability import FreqDist
# import re
# stopList = []
# for a, b in train_split:
#   for word in b:
#     stopList.append(word)
    
# fdist = FreqDist(stopList)
# top_ten = fdist.most_common(15)
# top_ten_words = [x for x, y in top_ten]
# def remove_stop_words(old_data):
#   temp_list_new = []
#   temp_list_labels = []
#   for a, sentence in old_data:
#     temp_list = []
#     #sentence_new = clean_str(' '.join(sentence))
#     sentence_new = sentence
#     for word in sentence_new:
#       if word not in top_ten_words:
#         #print(word)
#         temp_list.append(word)
#       # if word == 'úol':
#       #   print("There")
#     temp_list_sent = " ".join(temp_list).strip()
#     temp_list_new.append(temp_list_sent)
#     temp_list_labels.append(a)
#   return temp_list_new, temp_list_labels
# # Tokenization/string cleaning for dataset
# def clean_str(inp):
#     new_inp = re.sub(r'[^\w\s]', '', inp)
#     return new_inp.strip().lower().split(" ")
# #print(top_ten)
# #
# #print(top_ten_words)

from nltk.probability import FreqDist
import re
stopList = []
for a, b in train_split:
  for word in b:
    stopList.append(word)
    
fdist = FreqDist(stopList)
top_ten = fdist.most_common(15)
top_ten_words = [x for x, y in top_ten]
def remove_stop_words(old_data, word_index_local):
  temp_list_new = []
  temp_list_labels = []
  ctr = 0
  for a, sentence in old_data:
    temp_list = []
    #sentence_new = clean_str(' '.join(sentence))
    sentence_new = sentence
    for word in sentence_new:
      if word not in top_ten_words:
        #print(word)
        val = word_index_local.get(word, 0)
        if val == 0:
          ctr += 1
          #print(word)
        temp_list.append(word_index_local.get(word, 0))
      # if word == 'úol':
      #   print("There")
    temp_list_new.append(temp_list)
    temp_list_labels.append(a)
  print(ctr)
  return temp_list_new, temp_list_labels
# Tokenization/string cleaning for dataset
def clean_str(inp):
    new_inp = re.sub(r'[^\w\s]', '', inp)
    return new_inp.strip().lower().split(" ")
#print(top_ten)
#
#print(top_ten_words)

In [ ]:
print(dev_split)
train_text_new, train_label_new = remove_stop_words(train_split, word_index)
dev_text_new, dev_label_new = remove_stop_words(dev_split, word_index)
print(dev_text_new)
# stopList2 = []
# for a, b in train_split_new:
#   for word in b:
#     stopList2.append(word)
# fdist2 = FreqDist(stopList2)
# top_ten_2 = fdist2.most_common(10)
# print(top_ten_2)
# print(len(train_split_new))

In [ ]:

import os
import re

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from keras import backend as K
from keras.models import Model
from keras import initializers
from keras.layers import Layer
from keras.layers import Dense, Input
from keras.layers import Embedding, GRU, Bidirectional, TimeDistributed
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils.np_utils import to_categorical
from nltk import tokenize
from keras.callbacks import ReduceLROnPlateau

In [ ]:

from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
x_train = np.expand_dims(np.array(pad_sequences(train_text_new, maxlen=MAX_SEQUENCE_LENGTH, 
                     padding="post", truncating="post")), axis=1)
y_train = to_categorical(np.array(train_label_new))
x_val = np.expand_dims(pad_sequences(dev_text_new, maxlen=MAX_SEQUENCE_LENGTH, 
                     padding="post", truncating="post"), axis=1)
y_val = to_categorical(np.array(dev_label_new))
print(x_train.shape)
print([x_train[0]])
print(x_val.shape)
print(type(y_val))

In [ ]:
print(dev_split)
train_text_new_2, train_label_new_2 = remove_stop_words(train_split, word_index_2)
dev_text_new_2, dev_label_new_2 = remove_stop_words(dev_split, word_index_2)
print(dev_text_new_2)

In [ ]:
from keras.preprocessing.sequence import pad_sequences
x_train_2 = np.expand_dims(pad_sequences(train_text_new, maxlen=MAX_SEQUENCE_LENGTH, 
                     padding="post", truncating="post"), axis=1)
x_val_2 = np.expand_dims(pad_sequences(dev_text_new_2, maxlen=MAX_SEQUENCE_LENGTH, 
                     padding="post", truncating="post"), axis=1)

y_train_2= np.array(to_categorical(train_label_new_2))
y_val_2 = np.array(to_categorical(dev_label_new_2))

In [ ]:
# Implementation of Hierarchical Attentional Networks for Document Classification
# http://www.cs.cmu.edu/~./hovy/papers/16HLT-hierarchical-attention-networks.pdf


maxlen = MAX_SEQUENCE_LENGTH
max_sentences = 1
max_words = 50000
embedding_dim = 100
# validation_split = 0.2
reviews = []
labels = []
texts = []
# glove_dir = "./glove.6B"
embeddings_index = {}


# class defining the custom attention layer
class HierarchicalAttentionNetwork(Layer):
    def __init__(self, attention_dim):
        #self.init = initializers.get('normal')
        self.init = initializers.get('glorot_normal')
        self.supports_masking = False
        self.attention_dim = attention_dim
        super(HierarchicalAttentionNetwork, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.b = K.variable(self.init((self.attention_dim,)))
        self._trainable_weights = [self.W, self.b, self.u]
        super(HierarchicalAttentionNetwork, self).build(input_shape)

    # def compute_mask(self, inputs, mask=None):
    #     return mask

    def call(self, x, mask=None):
       
        attention_input = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        attenition_op = K.exp(K.squeeze(K.dot(attention_input, self.u), -1))

        # Helps in finding attention of weight in the sentence
        attenition_op /= K.cast(K.sum(attenition_op, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        weighted_input = x * K.expand_dims(attenition_op)
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

# input_data = pd.read_csv('labeledTrainData.tsv', sep='\t')

# for idx in range(input_data.review.shape[0]):
#     text = BeautifulSoup(input_data.review[idx], features="html5lib")
#     text = clean_str(text.get_text().encode('ascii', 'ignore'))
#     texts.append(text)
#     sentences = tokenize.sent_tokenize(text)
#     reviews.append(sentences)
#     labels.append(input_data.sentiment[idx])




In [ ]:
# # Tokenization Takes place here
# # Create complete train + dev
# texts = train_text_new + dev_text_new

# tokenizer = Tokenizer(num_words=max_words, filters=']})\t\n', char_level=False)
# tokenizer.fit_on_texts(texts)

# word_index = tokenizer.word_index
# print('Total %s unique tokens.' % len(word_index))

# def tokenize_each_dataset(reviews):
#   data = np.zeros((len(texts), max_sentences, maxlen), dtype='int32')
#   for i, sentences in enumerate(reviews):
#     #print(sentences)
#     j = 0
#     sent = sentences
#     if j < max_sentences:
#       wordTokens = text_to_word_sequence(sent, filters=']})\t\n')
#       #print(wordTokens)
#       k = 0
#       for _, word in enumerate(wordTokens):
#         if word == 'úêê':
#           print('there' + word)
#           print(wordTokens)
#           print(sent)
#         if k < maxlen and tokenizer.word_index[word] < max_words:
#           #print(j)
#           data[i, j, k] = tokenizer.word_index[word]
#           k = k + 1
#   return data

# # labels = to_categorical(np.asarray(labels))
# # print('Shape of reviews (data) tensor:', data.shape)
# # print('Shape of sentiment (label) tensor:', labels.shape)

# # Tokenize dev and train set using x_val, y_val
# x_train = tokenize_each_dataset(train_text_new)
# y_train = to_categorical(np.asarray(train_label_new))

# x_val = tokenize_each_dataset(dev_text_new)
# y_val = to_categorical(np.asarray(dev_label_new))

# print(x_val)

In [ ]:
# Build the Embedding layer here
def get_weight_matrix(vocab, model_local):
    total = 0
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with random 
    embedding_matrix = np.random.random((vocab_size, embedding_dim))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        if len(word.strip()) > 0:
          try:
            embedding_matrix[i] = model_local.wv.get_vector(word.strip())
          except KeyError as e:
            total += 1
            #print(e)
    print(total)
    return embedding_matrix

embedding_matrix = get_weight_matrix(word_index, model)
embedding_matrix_2 = get_weight_matrix(word_index_2, model2)


In [ ]:

# indices = np.arange(data.shape[0])
# np.random.shuffle(indices)
# data = data[indices]
# labels = labels[indices]
# nb_validation_samples = int(validation_split * data.shape[0])

# x_train = data[:-nb_validation_samples]
# y_train = labels[:-nb_validation_samples]
# x_val = data[-nb_validation_samples:]
# y_val = labels[-nb_validation_samples:]

# print('Number of positive and negative reviews in training and validation set')
# print y_train.sum(axis=0)
# print y_val.sum(axis=0)


# f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# print('Total %s word vectors.' % len(embeddings_index))

# # building Hierachical Attention network
# embedding_matrix = np.random.random((len(word_index) + 1, embedding_dim))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector

print(maxlen)

embedding_layer = Embedding(len(word_index) + 1, embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=True)

def word_attention():
  sentence_input = Input(shape=(maxlen,), dtype='int32')
  sequence_emded_new = embedding_layer(sentence_input)
  fn_lstm_word = Bidirectional(GRU(100, return_sequences=True))(sequence_emded_new)
  fn_attn_word = HierarchicalAttentionNetwork(100)(fn_lstm_word)
  sentenceEncoder = Model(sentence_input, fn_attn_word)
  return sentenceEncoder

def sentence_attention(sentenceEncoder):
  complete_input = Input(shape=(max_sentences, maxlen), dtype='int32')
  review_encoder = TimeDistributed(sentenceEncoder)(complete_input)
  fn_lstm = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
  fn_attn = HierarchicalAttentionNetwork(100)(fn_lstm)
  compare_preds = Dense(2, activation='softmax')(fn_attn)
  model_han = Model(complete_input, compare_preds)
  return model_han

sentence_encode =  word_attention()
model_final = sentence_attention(sentenceEncoder=sentence_encode)
model_final.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.0001)

#print("model fitting - Hierachical attention network")
print('Trial Fitting.')
#model_final.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=20, batch_size=100, callbacks=[reduce_lr])

For Final Training

In [ ]:
# For Final Training
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,patience=5, min_lr=0.00001)
x_final = np.concatenate((x_train, x_val), axis=0)
y_final = np.concatenate((y_train, y_val), axis=0)
print(x_final.shape)
print(y_final.shape)
model_final.fit(x_final, y_final, validation_data=(x_val, y_val), epochs=20, batch_size=32, callbacks=[reduce_lr])

Make Predictions

In [ ]:
def remove_stop_words_test(old_data, word_index_local):
  temp_list_new = []
  temp_list_labels = []
  ctr = 0
  for sentence in old_data:
    temp_list = []
    #sentence_new = clean_str(' '.join(sentence))
    sentence_new = sentence
    for word in sentence_new:
      if word not in top_ten_words:
        #print(word)
        val = word_index_local.get(word, 0)
        if val == 0:
          ctr += 1
          #print(word)
        temp_list.append(word_index_local.get(word, 0))
      # if word == 'úol':
      #   print("There")
    temp_list_new.append(temp_list)
  print(ctr)
  return temp_list_new

def process_text_test(list_sentences):
  comments = []
  ctr = 0
  for text in list_sentences:
    ctr += 1
    #print(text[0])
    text_curr = text[0]
    single_sentence = ' '.join(text_curr).strip().lower()
    temp_text = single_sentence.replace('.', '')
    text = temp_text.replace(',', '')
    txt = tk.tokenize(text)
    comments.append(txt)
  print("Entries "+ctr)
  return comments

In [ ]:
test_split_processed = process_text_test(test_split)
test_split_new = remove_stop_words_test(test_split_processed, word_index)
x_pred = np.expand_dims(pad_sequences(test_split_new, maxlen=MAX_SEQUENCE_LENGTH, 
                     padding="post", truncating="post"), axis=1)

y_pred = model_final.predict(x_pred)
print(y_pred.shape)
y_classes = y_pred.argmax(axis=-1)
print(y_classes.shape)
print(y_classes)

In [ ]:

# # indices = np.arange(data.shape[0])
# # np.random.shuffle(indices)
# # data = data[indices]
# # labels = labels[indices]
# # nb_validation_samples = int(validation_split * data.shape[0])

# # x_train = data[:-nb_validation_samples]
# # y_train = labels[:-nb_validation_samples]
# # x_val = data[-nb_validation_samples:]
# # y_val = labels[-nb_validation_samples:]

# # print('Number of positive and negative reviews in training and validation set')
# # print y_train.sum(axis=0)
# # print y_val.sum(axis=0)


# # f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
# # for line in f:
# #     values = line.split()
# #     word = values[0]
# #     coefs = np.asarray(values[1:], dtype='float32')
# #     embeddings_index[word] = coefs
# # f.close()

# # print('Total %s word vectors.' % len(embeddings_index))

# # # building Hierachical Attention network
# # embedding_matrix = np.random.random((len(word_index) + 1, embedding_dim))
# # for word, i in word_index.items():
# #     embedding_vector = embeddings_index.get(word)
# #     if embedding_vector is not None:
# #         # words not found in embedding index will be all-zeros.
# #         embedding_matrix[i] = embedding_vector

# print(maxlen)

# embedding_layer = Embedding(len(word_index_2) + 1, embedding_dim, weights=[embedding_matrix_2], input_length=maxlen, trainable=True)

# sentence_input = Input(shape=(maxlen,), dtype='int32')
# embedded_sequences = embedding_layer(sentence_input)
# lstm_word = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
# attn_word = HierarchicalAttentionNetwork(100)(lstm_word)
# sentenceEncoder = Model(sentence_input, attn_word)

# review_input = Input(shape=(max_sentences, maxlen), dtype='int32')
# review_encoder = TimeDistributed(sentenceEncoder)(review_input)
# lstm_sentence = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
# attn_sentence = HierarchicalAttentionNetwork(100)(lstm_sentence)
# preds = Dense(2, activation='softmax')(attn_sentence)
# model_final = Model(review_input, preds)

# model_final.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# print("model fitting - Hierachical attention network")
# model_final.fit(x_train_2, y_train_2, validation_data=(x_val_2, y_val_2), epochs=10, batch_size=64)

In [ ]:
# Eventually, results need to be a list of 2028 0 or 1's
results = y_classes

### Output Prediction Result File

You will need to submit a prediction result file. It should have 2028 lines, every line should be either 0 or 1, which is your model's prediction on the respective test set instance.

In [ ]:
# suppose you had your model's predictions on the 2028 test cases read from test_enc_unlabeled.tsv, and 
#those results are in the list called 'results'
assert (len(results) == 2028)

In [ ]:
# make sure the results are not float numbers, but intergers 0 and 1
results = [int(x) for x in results]

In [ ]:
# write your prediction results to 'upload_predictions.txt' and upload that later
with open('upload_predictions.txt', 'w', encoding = 'utf-8') as fp:
    for x in results:
        fp.write(str(x) + '\n')